In [255]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import matplotlib.pyplot as plt



In [256]:
df = pd.read_csv(f'../data/market_integrates_100.csv')
print(df)


     market_ids  firm_ids  characteristic1  characteristic2     prices  \
0             1         1         5.433865         2.042325  23.994561   
1             1         2         5.907737         2.412709  22.799957   
2             1         3         5.943911         2.174390  22.293546   
3             1         4         5.329441         2.844997  24.472704   
4             1         5         5.083033         2.137079  25.714607   
..          ...       ...              ...              ...        ...   
995         100         6         5.360283         2.580602  22.145922   
996         100         7         5.958183         2.650134  21.172652   
997         100         8         5.184745         2.346844  26.109766   
998         100         9         5.223118         2.464959  25.816435   
999         100        10         5.019838         2.675947  23.031981   

     marginal_cost    shares   profits   markups  e_quantity    capital  \
0        16.639639  0.057343  0.4217

In [257]:
# theta_0 = np.array([2, -0.5, 0.3, 0.05, 1.])
theta_0 = np.array([1., -1., -1., 1., 1.])




In [258]:
def simulate_market_shares_per_period(theta, df, v_p, n_consumers, n_firms, t): 
    beta = theta[:3]
    mu = theta[3]
    omega = theta[4]
    X0 = np.ones(n_firms)
    X1 = df.characteristic1.head(n_firms)
    X2 = df.characteristic2.head(n_firms)
    X_car = np.column_stack((X1, X2))
    X = np.column_stack((X0, X_car))

    X_for_utility = np.repeat(X, n_consumers, axis=0)
    price_r = np.reshape(np.array(df.predict_prices[t*n_firms:(t+1)*n_firms]), (1, n_firms))
    alpha_i = np.reshape(-(np.exp(mu + omega*v_p)), (n_consumers, 1))
    random_coeff = np.ravel((alpha_i*price_r).T)

    u = X_for_utility@beta + random_coeff
    u_r = np.reshape(u, (n_firms, n_consumers))
    sum_u = np.sum(np.exp(u_r))

    all_probs = np.exp(u_r)/(1 + sum_u)
    market_shares = np.sum(all_probs, axis=1)
    
    return market_shares

In [259]:
def f(theta, df, n_consumers, n_firms, T):
    true_share = df.shares
    all_est_shares = np.zeros(T*n_firms)
    v_p_all = np.random.normal(0, 1, n_consumers*T)
    for t in range(0, T-1):
        v_p = v_p_all[t*n_consumers:(t+1)*n_consumers]
        estimates_shares_market = simulate_market_shares_per_period(theta, df, v_p, n_consumers, n_firms, t)
        all_est_shares[t*n_firms:(t+1)*n_firms] = estimates_shares_market
    return np.linalg.norm(true_share - all_est_shares)

In [260]:
from sklearn.linear_model import LinearRegression

In [261]:
x = df[['marginal_cost', 'characteristic1', 'characteristic2']]
y = df['prices']

res = LinearRegression().fit(x, y)

print('Intercept: \n', res.intercept_)
print('Coefficients: \n', res.coef_)

predictions_price = res.predict(x)
print(predictions_price)
df['predict_prices'] = predictions_price
print(df)



Intercept: 
 -3.450629655708763
Coefficients: 
 [1.49223383 0.55183687 0.23203127]
[24.85209216 23.68939277 22.99508744 25.63198948 26.92705295 24.61940545
 21.1720073  23.58334648 24.61203304 25.73962896 22.62191464 21.53161151
 21.73457599 23.72902483 24.07683981 22.85747538 19.49083892 22.27847205
 22.82522878 23.38562297 24.8893637  23.03450523 23.22957713 25.31555814
 25.47126939 24.63057282 21.21240093 23.85255841 24.24355747 24.69511336
 28.01105202 26.8569362  26.08365242 29.8869489  31.14870301 26.81144268
 23.47646671 27.38058717 28.0578754  29.02373614 25.85044727 25.56097197
 25.12986321 28.4428503  29.6773378  24.9381063  22.26101883 24.3711694
 25.46530661 26.96697521 19.77056212 19.78555412 19.41195496 21.61252709
 21.88179733 19.57956428 17.80320421 19.2560683  19.44065073 19.74370785
 26.43954631 23.86185568 23.86774912 26.95051658 27.29164401 24.61059642
 21.51753316 23.90155133 23.67658518 24.5123885  25.77848314 22.59709355
 22.41492054 25.94490485 26.44588288 23.89

In [262]:
N = 500 
firms  = 10
time = 100 

all_val_b0 = []
all_val_b1 = []
all_val_b2 = []
all_val_mu = []
all_val_omega = []

for l in range(1, 20):
    
    res = minimize(f, theta_0, args=(df, N, firms, time), method = 'Nelder-Mead')
    optim_vec = res.x

    all_val_b0.append(optim_vec[0])
    all_val_b1.append(optim_vec[1])
    all_val_b2.append(optim_vec[2])
    all_val_mu.append(optim_vec[3])
    all_val_omega.append(optim_vec[4])


df_estimation_values = pd.DataFrame({'beta_0': all_val_b0,
                                    'beta_1': all_val_b1,
                                    'beta_2': all_val_b2, 
                                    'mu': all_val_mu, 
                                    'omega': all_val_omega, 
                                    })
df_estimation_values.to_csv(f'../data/first_estimation_market_inter_100', index=False)


In [263]:
print(df_estimation_values)

      beta_0    beta_1    beta_2        mu     omega
0   1.249722 -0.366363 -0.737717  1.086881  1.482167
1   1.255717 -0.299379 -1.069252  0.874553  1.427256
2   1.072319 -0.347188 -1.165296  0.777822  1.572818
3   0.912010 -0.252011 -1.097421  0.937900  1.500965
4   1.393117 -0.634082 -0.844923  0.801772  1.815885
5   1.245285 -0.579621 -0.890646  0.394660  1.535278
6   1.249066 -0.624085 -0.788521  0.607729  1.658238
7   1.260843 -0.423236 -0.869762  0.880682  1.508900
8   1.287465 -0.564376 -0.773842  0.720578  1.604980
9   1.184558 -0.518869 -0.881946  0.605004  1.550402
10  1.252821 -0.564189 -0.696227  0.800331  1.607669
11  1.189842 -0.632694 -0.566816  0.858677  1.648138
12  1.171163 -0.579552 -0.898983  0.569855  1.649198
13  1.441509 -0.564670 -0.843873  0.500598  1.479976
14  1.264299 -0.548272 -0.791639  0.728704  1.583924
15  1.304832 -0.486984 -1.127286  0.491817  1.590798
16  1.059443 -0.491179 -0.511720  1.113122  1.576400
17  1.164883 -0.615258 -0.526684  0.923174  1.